# Práctica 1 - Self-Organising Maps - COLORES
## Preparación de entorno
#### Importar librerías de código

In [1]:
# from __future__ import division

import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from matplotlib import patches as patches

%matplotlib inline

#### Dataset que se va a utilizar para el entrenamiento

In [2]:
# Código para obtener el Dataset que se va a usar en el entrenamiento
file= r'C:\Users\Festr\OneDrive\Escritorio\IA 2\Laboratorio 1\Jugadores_Característica.csv'
datos = pd.read_csv(file)
datosNoId=datos.drop(datos.columns[0], axis = 1, inplace = True)


s="12+1"

f,g=s.split('+')
print(int(f)+int(g))
datos.head()
dt=datos.to_numpy()
len(dt[1][3])
dt[1][3]

13


C:\Users\Festr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (11,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


'95'

In [3]:
dt=datos.to_numpy()
nombres=[]
aux1 = 0.0
aux2=0.0
mini=99
maxi=0
for col in datos.columns: 
    nombres.append(col)
for i in range(dt.shape[0]):
    for j in range(dt.shape[1]):
        if isinstance(dt[i][j],str):
            if len(dt[i][j])>2:
                if dt[i][j][2] == '+':
                    a,b= dt[i][j].split('+')
                    dt[i][j]=int(a)+int(b)
                elif dt[i][j][2]=='-':
                    a,b= dt[i][j].split('-')
                    dt[i][j]=int(a)-int(b)
            dt[i][j]=pd.to_numeric(dt[i][j])
        if dt[i][j]> maxi:
            maxi=dt[i][j]
        if dt[i][j]< mini:
            mini=dt[i][j]
print(maxi,mini) 


241219 -5


## SOM Setup
#### Variables definidas por el alumno

In [4]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
# Inicializa normalizar_datos dependiendo de si tienes que normalizar los datos o no
lado_mapa = 15
periodo = 500
learning_rate = 0.06
normalizar_datos = True
dt

array([[89, 63, 89, ..., 80, 85, 88],
       [92, 48, 90, ..., 59, 90, 85],
       [94, 56, 96, ..., 53, 80, 83],
       ...,
       [66, 26, 60, ..., 32, 37, 33],
       [57, 46, 58, ..., 46, 51, 25],
       [48, 52, 49, ..., 67, 22, 19]], dtype=object)

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [5]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.
num_entradas = 35
num_datos = dt.shape[0]

# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario = lado_mapa/2

def minmax_norm(datos_input): 
    return (datos_input - mini) / ( maxi - mini)
## Normaliza los datos si fuese necesario dividiendo cada dato por el máximo en la matriz
if normalizar_datos:
    dt = minmax_norm(dt)
    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
matriz_pesos = np.random.random((lado_mapa,lado_mapa,35))
print(matriz_pesos.shape)


(15, 15, 35)


#### Funciones para entrenar/clasificar

In [6]:
# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""
def calcular_bmu(patron_entrada, m_pesos, m):
    MenorDist=0
    MejorVector=[0,0,0]
    a=[]
    Resultado=0
    idx=[]
    
    for i in range(m-1):
        for j in range(m-1):
            
            a=m_pesos[i][j]
            Resultado=distancia(patron_entrada, a)
            if i == 0 and j==0 :
                MenorDist=Resultado
                MejorVector=m_pesos[i][j]
                idx=[i,j]
            else:
                #print(MenorDist, Resultado)
                if MenorDist>Resultado:
                    #print(m_pesos[i][j])
                    MenorDist=Resultado
                    MejorVector=m_pesos[i][j]
                    idx=[i,j]
    if len(idx)<2:
        print("patron: ", patron_entrada)
        print("coor: ",idx)
        print("Peso: ",MejorVector,"i,j: ", i,j)
        print("Distancia: ",MenorDist)
        
    #print("BMU: ", MejorVector, "Index: ", idx)
    return MejorVector,idx
#a,b=calcular_bmu(dt[6], matriz_pesos, lado_mapa)


In [7]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
    
    #lr_final = lr_inicial * (1 - i/n_iteraciones) 
    learning_rate = lr_inicial + (-lr_inicial/n_iteraciones)*i
    return learning_rate

In [8]:
#Función para calcular la distancia euclidea
def distancia(patron, peso):
    distancia=0
    comp=0
    for i in range(dt.shape[1]):
        comp+=(peso[i]-patron[i])**2
    distancia=(comp)**0.5
    return distancia


In [9]:
def ajuste_Pesos(i,j,pesos):
    total=0
    for a in range(num_entradas):
        total+=pesos[i][j][a]**2
    return (total)**0.5

In [10]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
    
    vecindario_i = 1 + vecindario_inicial * (1 - i/n_iteraciones)
    return vecindario_i



In [11]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU**2 / (2*vecindario_actual**2))

#### Funciones para dibujar la salida de la red

In [12]:
# Función para pintar una matriz de valores como colores RGB
def pintar_mapa(matriz_valores, iteracion):
    fig = plt.figure()
    
    # Establece ejes
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim((0, matriz_pesos.shape[0]+1))
    ax.set_ylim((0, matriz_pesos.shape[1]+1))
    ax.set_title('Self-Organising Map después de %d iteraciones' % iteracion)

    # Dibuja los rectángulos de color RGB
    for x in range(1, matriz_valores.shape[0] + 1):
        for y in range(1, matriz_valores.shape[1] + 1):
            #print(matriz_valores[x-1,y-1,:])
            ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1,
                         facecolor=matriz_valores[x-1,y-1,:],
                         edgecolor='none'))
    plt.show()


## SOM Entrenamiento

In [13]:
# Entrena la red con el dataset de entrenamiento

for i in range(periodo):
    azar=np.random.randint(0, len(dt))
    patron=dt[azar]
    BMU, coordenadas = calcular_bmu(patron, matriz_pesos, lado_mapa)#encontramos la bmu y sus coordenadas
    eta=variacion_learning_rate(learning_rate, i+1, periodo)#calculamos eta
    v=variacion_vecindario(vecindario, i+1, periodo)#calculamos el vecindario
    
    #comprobamos cuales son las neuronas del vecindario de la BMU
    for w in range(len(matriz_pesos)):
        for j in range(len(matriz_pesos)):
            deu= ((w-coordenadas[0])**2 + (j-coordenadas[1])**2 )**0.5
            
            if deu <= v :
                amortiguacion= decay(deu, v)
                matriz_pesos[w][j]= matriz_pesos[w][j]/ajuste_Pesos(w,j,matriz_pesos)
                matriz_pesos[w][j]= matriz_pesos[w][j] + eta*amortiguacion*(patron-matriz_pesos[w][j])
    if i%100==0:
        print("Vuelta nº: ", i)
matriz_pesos  

Vuelta nº:  0
Vuelta nº:  100
Vuelta nº:  200
Vuelta nº:  300
Vuelta nº:  400


array([[[0.15316958, 0.07268904, 0.20311814, ..., 0.13546165,
         0.19848832, 0.00425877],
        [0.15682342, 0.18443062, 0.15475325, ..., 0.10773715,
         0.23332618, 0.01065518],
        [0.00372416, 0.21698143, 0.08340447, ..., 0.15916906,
         0.01601571, 0.04832374],
        ...,
        [0.04019634, 0.07736544, 0.02011472, ..., 0.07764801,
         0.0682289 , 0.0679678 ],
        [0.10354601, 0.10093931, 0.03387815, ..., 0.19707631,
         0.11316788, 0.10901799],
        [0.01180787, 0.21472507, 0.12673093, ..., 0.10412608,
         0.01304437, 0.05665229]],

       [[0.05475199, 0.27406335, 0.22570597, ..., 0.32975008,
         0.33131059, 0.12960569],
        [0.04478758, 0.15747599, 0.12654771, ..., 0.01317625,
         0.13297339, 0.04106854],
        [0.14486081, 0.00301316, 0.12846489, ..., 0.09682097,
         0.11098494, 0.10401481],
        ...,
        [0.00882705, 0.00680648, 0.00429589, ..., 0.00964569,
         0.001425  , 0.00967413],
        [0.0

In [14]:
#Función para calcular la distancia euclidea

def errorCuantificacion(dis):
    contador = 0;
    error_cuantificacion = 0;
    
    for j in range(dis.shape[0]):
        for k in range(dis.shape[1]):
            if dis[j][k] != 0:
                contador+=1
                error_cuantificacion += dis[j][k]
                
    error_cuantificacion = error_cuantificacion / contador
    return error_cuantificacion

def errorTopologico(bmu1, bmu2):
    if ((bmu2[0] == bmu1[0] + 1 or bmu2[0] == bmu1[0] - 1) and bmu2[1] == bmu1[1]) or (bmu2[0] == bmu1[0] and (bmu2[1] == bmu1[1] + 1 or bmu2[1] == bmu1[1] - 1)) or ((bmu2[0] == bmu1[0] - 1) and (bmu2[1] == bmu1[1] + 1 or bmu2[1] == bmu1[1] - 1)) or ((bmu2[0] == bmu1[0] + 1) and (bmu2[1] == bmu1[1] + 1 or bmu2[1] == bmu1[1] - 1)):
        v = 0
    else:
        v = 1
    return v

def calcular2bmu(patron_entrada, m_pesos, m):
    MenorDist = 999
    MenorDist2 = 999
    MejorVector  =  [0,0,0]
    MejorVector2  =  [0,0,0]
    a=[]
    Resultado = 0
    idx=[]
    idx2=[]
    
    for i in range(m-1):
        for j in range(m-1):
            a = m_pesos[i][j]
            Resultado = distancia(patron_entrada,a)
            
            if MenorDist2 > Resultado and Resultado > MenorDist:
                MenorDist2 = Resultado
                MejorVector2 = m_pesos[i][j]
                idx2 = [i,j]

            elif MenorDist > Resultado: 
                MenorDist = Resultado
                MejorVector = m_pesos[i][j]
                idx = [i,j]
     
    return MejorVector2, idx2



## SOM Clasificación

In [ ]:
clasificacion = np.zeros((lado_mapa, lado_mapa, 35))
activacion = np.zeros((lado_mapa, lado_mapa))
distancias = np.zeros((lado_mapa, lado_mapa))
suma = 0
clase = 0
eTopo = 0


for i in range(dt.shape[0]):
    BMU, coor = calcular_bmu(dt[i], matriz_pesos, lado_mapa)
    BMU2, coor2 = calcular2bmu(dt[i], matriz_pesos, lado_mapa)
    clasificacion[coor[0]][coor[1]] = dt[i]
    activacion[coor[0]][coor[1]] += 1
    distancias[coor[0]][coor[1]] = distancias[coor[0]][coor[1]] + distancia(dt[i], BMU) 
    eTopo += errorTopologico(coor, coor2)
    if i%100==0:
        print("Vueltas:", i)
    
for g in range(activacion.shape[0]): 
    for t in range(activacion.shape[1]):
        if  activacion[g][t] > 0:
            clase+=1
            distancias[g][t]=distancias[g][t]/activacion[g][t] 
      
            
nclases = clase
mapa_activaciones = activacion
mapa_distancia = distancias
eCuanti = errorCuantificacion(distancias)
eTopo = eTopo/100

mapa_distancia, mapa_activaciones, clase, eTopo, eCuanti

Vueltas: 0
Vueltas: 100
Vueltas: 200
Vueltas: 300
Vueltas: 400
Vueltas: 500
Vueltas: 600
Vueltas: 700
Vueltas: 800
Vueltas: 900
Vueltas: 1000
Vueltas: 1100
Vueltas: 1200
Vueltas: 1300
Vueltas: 1400
Vueltas: 1500
Vueltas: 1600
Vueltas: 1700
Vueltas: 1800
Vueltas: 1900
Vueltas: 2000
Vueltas: 2100
Vueltas: 2200
Vueltas: 2300
Vueltas: 2400
Vueltas: 2500
Vueltas: 2600
Vueltas: 2700
Vueltas: 2800
Vueltas: 2900
Vueltas: 3000
Vueltas: 3100
Vueltas: 3200
Vueltas: 3300
Vueltas: 3400
Vueltas: 3500
Vueltas: 3600
Vueltas: 3700
Vueltas: 3800
Vueltas: 3900
Vueltas: 4000
Vueltas: 4100
Vueltas: 4200
Vueltas: 4300
Vueltas: 4400
Vueltas: 4500
Vueltas: 4600
Vueltas: 4700
Vueltas: 4800
Vueltas: 4900
Vueltas: 5000
Vueltas: 5100
Vueltas: 5200
Vueltas: 5300
Vueltas: 5400
Vueltas: 5500
Vueltas: 5600
Vueltas: 5700
Vueltas: 5800
Vueltas: 5900
Vueltas: 6000
Vueltas: 6100
Vueltas: 6200
Vueltas: 6300
Vueltas: 6400
Vueltas: 6500
Vueltas: 6600
Vueltas: 6700
Vueltas: 6800
Vueltas: 6900
Vueltas: 7000
Vueltas: 7100
Vuel

## SOM Prueba

In [ ]:
# Clasifica nuevos patrones
nclases = clase
mapa_activaciones = activacion
mapa_distancia = distancias
eTopo = eTopo*100/dt.shape[0]

mapa_distancia, mapa_activaciones, clase, eTopo, eCuanti

In [ ]:
hola=[]
hola.append([coor,1])
len(activaciones)